In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_ibm_runtime import (QiskitRuntimeService,Session,EstimatorV2,EstimatorOptions)


In [ ]:
def b92_estimator_simulation(n_bits=1000, theta_degrees=25, test_fraction=0.2,seed=42,backend=None):
    np.random.seed(seed)
    # 1) Define B92 states |u0>, |u1>
    theta = np.deg2rad(theta_degrees)
    # |u0> = |0> in computational basis
    # |u1> = cos(theta)|0> + sin(theta)|1>
    u0 = np.array([1.0, 0.0], dtype=complex)
    u1 = np.array([np.cos(theta), np.sin(theta)], dtype=complex)
    # 2) Define P0 = I - |u1><u1|, P1 = I - |u0><u0|
    I2 = np.eye(2, dtype=complex)
    proj_u1 = np.outer(u1, np.conjugate(u1))  # |u1><u1|
    proj_u0 = np.outer(u0, np.conjugate(u0))  # |u0><u0|
    P0_mat = I2 - proj_u1
    P1_mat = I2 - proj_u0
    P0_op = SparsePauliOp.from_operator(P0_mat)
    P1_op = SparsePauliOp.from_operator(P1_mat)

    # 3) Random bit preparation (Alice) and measurement choice (Bob)
    alice_bits = np.random.randint(0, 2, size=n_bits)
    bob_measurements = np.random.randint(0, 2, size=n_bits)

    # We'll build circuits (one per qubit) + corresponding operators
    circuits = []
    operators = []

    for i in range(n_bits):
        bit = alice_bits[i]
        meas_choice = bob_measurements[i]
        qc = QuantumCircuit(1)
        if bit == 1:
            qc.ry(2 * theta, 0)
        if meas_choice == 0:
            operators.append(P0_op)
        else:
            operators.append(P1_op)

        circuits.append(qc)

    estimator = AerEstimator()
    job = estimator.run(circuits=circuits, observables=operators, shots=1024)
    p_positive = job.result().values  # Probability of "positive" outcome
    final_key = []
    for i in range(n_bits):
        prob_plus = p_positive[i].real
        if prob_plus < 0: 
            prob_plus = 0
        elif prob_plus > 1:
            prob_plus = 1
        
        outcome = np.random.choice(["+", "-"], p=[prob_plus, 1 - prob_plus])
        if outcome == "+":
            final_key.append(alice_bits[i])

    final_key = np.array(final_key, dtype=int)

    sample_size = int(test_fraction * len(final_key))
    if sample_size < 1:
        return final_key.tolist(), None

    test_indices = np.random.choice(len(final_key), size=sample_size, replace=False)
    alice_test = final_key[test_indices]
    bob_test = final_key[test_indices] 


    mismatches = np.count_nonzero(alice_test != bob_test)
    qber = mismatches / sample_size

    mask = np.ones(len(final_key), dtype=bool)
    mask[test_indices] = False
    final_key_after_test = final_key[mask]

    return final_key_after_test.tolist(), qber

if __name__ == "__main__":
    raw_key, qber_value = b92_estimator_simulation(n_bits=1000, theta_degrees=25, test_fraction=0.2)
    print("Local Simulation Results:")
    print("  Raw key length after QBER test:", len(raw_key))
    print("  Sample of key bits:", raw_key[:20])
    if qber_value is not None:
        print(f"  QBER: {qber_value:.2%}")
    else:
        print("  Not enough bits to calculate QBER.")

C:\Users\utkar\AppData\Local\Temp\ipykernel_18280\1372965234.py:137: DeprecationWarning: Estimator has been deprecated as of Aer 0.15, please use EstimatorV2 instead.
  raw_key, qber_value = b92_estimator_simulation(n_bits=1000, theta_degrees=25, test_fraction=0.2)
C:\Users\utkar\AppData\Local\Temp\ipykernel_18280\1372965234.py:137: DeprecationWarning: Option approximation=False is deprecated as of qiskit-aer 0.13. It will be removed no earlier than 3 months after the release date. Instead, use BackendEstimator from qiskit.primitives.
  raw_key, qber_value = b92_estimator_simulation(n_bits=1000, theta_degrees=25, test_fraction=0.2)


Local Simulation Results:
  Raw key length after QBER test: 78
  Sample of key bits: [0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1]
  QBER: 0.00%


In [ ]:

def generate_b92_observables(transpiled_circuits, bob_bases, theta_degrees=25):
    theta = np.deg2rad(theta_degrees)
    u1 = np.array([np.cos(theta), np.sin(theta)], dtype=complex)
    u0 = np.array([1.0, 0.0], dtype=complex)
    I = np.eye(2, dtype=complex)

    proj_u1 = np.outer(u1, u1.conj())
    proj_u0 = np.outer(u0, u0.conj())

    P0 = I - proj_u1  
    P1 = I - proj_u0  

    observables = []
    for qc, basis in zip(transpiled_circuits, bob_bases):
        q_idx = qc.find_bit(qc.qubits[0]).index

        op_matrix = P0 if basis == 0 else P1
        op_1qubit = SparsePauliOp.from_operator(op_matrix)

        label = ["I"] * qc.num_qubits
        op = SparsePauliOp.from_operator(op_matrix).tensorpower(1)
        full_op = op.expand([SparsePauliOp.from_operator(np.eye(2))] * qc.num_qubits)

        from qiskit.opflow import PauliOp, I, X, Y, Z

        factors = []
        for i in range(qc.num_qubits):
            if i == q_idx:
                factors.append(SparsePauliOp.from_operator(op_matrix))
            else:
                factors.append(SparsePauliOp.from_operator(I))

        full = factors[0]
        for f in factors[1:]:
            full = full.tensor(f)

        observables.append(full)

    return observables


In [5]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2, Session, EstimatorOptions


In [6]:


def generate_b92_observables(transpiled_circuits, bob_bases, theta_degrees=25):
    theta = np.deg2rad(theta_degrees)

    # Define states |u0> and |u1>
    u0 = np.array([1.0, 0.0], dtype=complex)
    u1 = np.array([np.cos(theta), np.sin(theta)], dtype=complex)

    # Projectors
    proj_u0 = np.outer(u0, u0.conj())
    proj_u1 = np.outer(u1, u1.conj())

    I2 = np.eye(2, dtype=complex)
    P0 = I2 - proj_u1
    P1 = I2 - proj_u0

    observables = []

    for qc, basis in zip(transpiled_circuits, bob_bases):
        num_qubits = qc.num_qubits
        q_idx = qc.find_bit(qc.qubits[0]).index

        projector = P0 if basis == 0 else P1
        single_op = SparsePauliOp.from_operator(projector)

        if num_qubits == 1:
            full_op = single_op
        else:
            identity = SparsePauliOp.from_operator(np.eye(2))
            ops = [single_op if i == q_idx else identity for i in range(num_qubits)]
            full_op = ops[0]
            for op in ops[1:]:
                full_op = full_op.tensor(op)

        observables.append(full_op)

    return observables


In [ ]:


def b92_ibm_hardware_simulation(backend, n_bits=100, theta_degrees=25, test_fraction=0.2, seed=42):
    np.random.seed(seed)

    theta = np.deg2rad(theta_degrees)
    alice_bits = np.random.randint(0, 2, n_bits)
    bob_bases = np.random.randint(0, 2, n_bits)

    circuits = []
    for bit in alice_bits:
        qc = QuantumCircuit(1)
        if bit == 1:
            qc.ry(2 * theta, 0)  # prepares |u1>
        circuits.append(qc)

    pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)
    transpiled_circuits = [pass_manager.run(circ) for circ in circuits]

    observables = generate_b92_observables(transpiled_circuits, bob_bases, theta_degrees)

    options = EstimatorOptions(
        resilience_level=1,
        dynamical_decoupling={"enable": True, "sequence_type": "XY4"}
    )

    with Session(backend=backend) as session:
        estimator = EstimatorV2(options=options)
        job = estimator.run(list(zip(transpiled_circuits, observables)))
        result = job.result()
    
        values = [float(r.data.evs) for r in result]


    final_key = []
    for i in range(n_bits):
        p_plus = values[i].real
        p_plus = min(max(p_plus, 0), 1)  # clamp
        outcome = np.random.choice(["+", "-"], p=[p_plus, 1 - p_plus])
        if outcome == "+":
            final_key.append(alice_bits[i])

    final_key = np.array(final_key)

    sample_size = int(test_fraction * len(final_key))
    if sample_size < 1:
        return final_key.tolist(), None

    indices = np.random.choice(len(final_key), sample_size, replace=False)
    test_bits = final_key[indices]
    qber = np.count_nonzero(test_bits != test_bits) / sample_size  # always 0 in ideal case

    mask = np.ones(len(final_key), dtype=bool)
    mask[indices] = False
    final_key = final_key[mask]

    return final_key.tolist(), qber

In [ ]:
 service = QiskitRuntimeService(channel="ibm_quantum",  # ibm_cloud 
                                token = 'ee14d25b26ea06305630f2a7e6eb2c0bbcf44295cbf50c0dbeb302280959c7c8573b573e393218eed994456c7c4799bfbb38cf52ddf85c1ba01cd00f83483b75')
 QiskitRuntimeService.save_account(channel='ibm_quantum', overwrite=True,
                                   token = 'ee14d25b26ea06305630f2a7e6eb2c0bbcf44295cbf50c0dbeb302280959c7c8573b573e393218eed994456c7c4799bfbb38cf52ddf85c1ba01cd00f83483b75')
backend = QiskitRuntimeService().least_busy(simulator=False, operational=True, min_num_qubits=100)
backend.num_qubits

127

In [9]:

key, qber = b92_ibm_hardware_simulation(backend=backend, n_bits=100, theta_degrees=25)

print("Final key:", key)
print("Key length:", len(key))
print("QBER:", f"{qber:.2%}" if qber is not None else "N/A")

c:\Users\utkar\anaconda3\envs\myenv\lib\site-packages\qiskit_ibm_runtime\qiskit_runtime_service.py:857: UserWarning: Your current pending jobs are estimated to consume 785.8086324424534 quantum seconds, but you only have 55 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)


Final key: [0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1]
Key length: 11
QBER: 0.00%
